#### Samiksya Subedi
#### Temperature data interpolation
#### This notebook creates 3 interpolation methods in temperature data and determines best interpolation methods. 

In [1]:
import arcpy
import requests
import zipfile
import os
import json
import pandas as pd
import psycopg2
from psycopg2 import sql
from io import StringIO
import arcgis
from arcgis.features import GeoAccessor, GeoSeriesAccessor


In [3]:
# Define Variables and paths
output_path = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"
working_directory = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"
db_connection = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"
feature_class_name = "mn_temp_stations_shp"  # Corrected variable name


### Creating a subset of the original temperature data

In [4]:
arcpy.conversion.FeatureClassToShapefile(
    Input_Features="mn_temp_stations_shp",
    Output_Folder=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab3'>

In [5]:
import arcpy
import numpy as np

# Set workspace environment
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"

# Input feature class
input_features = "mn_temp_stations_shp"

# Output folder
output_folder = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"

# Convert feature class to shapefile
arcpy.conversion.FeatureClassToShapefile(input_features, output_folder)

# Count the total number of features in the input feature class
total_count = int(arcpy.management.GetCount(input_features)[0])

# Number of features to select
num_to_select = 25  # Change this number according to your requirement

# Generate random object IDs to select
rand_ids = [str(i) for i in range(1, total_count + 1)]
selected_ids = set(np.random.choice(rand_ids, num_to_select, replace=False))

# Create a query to select features with selected object IDs
query = f"OBJECTID IN ({','.join(selected_ids)})"

# Create a new feature layer with selected features
subset_feature_layer = arcpy.management.MakeFeatureLayer(input_features, "subset_feature_layer", query)



### Interpolation

In [16]:
# Interpolate the temperature using 3 methods: IDW, Kriging, and GPI

arcpy.ddd.Idw(
    in_point_features="subset_feature_layer",
    z_field="max_tmpf",
    out_raster=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\MyProject21\MyProject21.gdb\Idw_mn_temp_1",
    cell_size=30000,
    power=2,
    search_radius="VARIABLE 12",
    in_barrier_polyline_features=None
)
arcpy.ddd.Kriging(
    in_point_features="subset_feature_layer",
    z_field="max_tmpf",
    out_surface_raster=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\MyProject21\MyProject21.gdb\Kriging_mn_t2",
    semiVariogram_props="Linear 0.021850 # # #",
    cell_size=500,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)
arcpy.ddd.Kriging(
    in_point_features="subset_feature_layer",
    z_field="max_tmpf",
    out_surface_raster=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\MyProject21\MyProject21.gdb\Kriging_mn_universal",
    semiVariogram_props="LinearDrift 0.021850 # # #",
    cell_size=1000,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\MyProject21\\MyProject21.gdb\\Kriging_mn_universal'>

### Accuracy assesment

In [13]:
arcpy.ga.ExploratoryInterpolation(
    in_features="subset_feature_layer",
    value_field="max_tmpf",
    out_cv_table=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\MyProject21\MyProject21.gdb\TempExploratoryInterpolation",
    out_arcpy.ddd.Kriging(
    in_point_features="subset_feature_layer",
    z_field="max_tmpf",
    out_surface_raster=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\MyProject21\MyProject21.gdb\Kriging_mn_t2",
    semiVariogram_props="Linear 0.021850 # # #",
    cell_size=500,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;ORDINARY_KRIGING;UNIVERSAL_KRIGING",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)


<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\MyProject21\\MyProject21.gdb\\TempExploratoryInterpolation'>

### Push accuracy assesment to geodatabase

In [21]:
import arcpy

# Set the workspace environment
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"

# Define the output table
out_table = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\MyProject21\MyProject21.gdb\TempExploratoryInterpolation"

# Define the output table name in the PostGIS database
table_name = "AccuracyAssessmentTable"

# Define connection parameters for PostGIS
db_connection = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"

# Copy the rows directly to the PostGIS database
arcpy.CopyRows_management(out_table, db_connection + "\\" + table_name)

# Print confirmation
print("Accuracy assessment table pushed to PostGIS.")


Accuracy assessment table pushed to PostGIS.


### Raster to points and send it to postgis 

In [14]:
# Step 2: Upload Points to SDE
#upload_points_to_sde(output_points, output_sde_connection, output_sde_feature_class)


In [17]:
import arcpy
import os

# Function to convert raster to point and push to database
def point_and_push(raster_name, local_gdb_path, geodatabase_path):
    # Convert raster to point data
    arcpy.conversion.RasterToPoint(
        in_raster=os.path.join(local_gdb_path, raster_name),
        out_point_features=os.path.join(local_gdb_path, raster_name + '_point'),
        raster_field="Value"
    )
    print(f"{raster_name} converted to point")

    # Push point data to remote database
    arcpy.conversion.FeatureClassToGeodatabase(
        os.path.join(local_gdb_path, raster_name + '_point'),
        geodatabase_path
    )
    print(f"{raster_name}_point pushed to database")

# Paths
local_gdb_path = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\MyProject21\MyProject21.gdb"
geodatabase_path = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"

# Interpolation raster names
interpolation_rasters = [
    "Idw_mn_temp_1",
    "Kriging_mn_t2",
    "Kriging_mn_universal"
]

# Convert each raster to point and push to the database
for raster_name in interpolation_rasters:
    point_and_push(raster_name, local_gdb_path, geodatabase_path)


Idw_mn_temp_1 converted to point
Idw_mn_temp_1_point pushed to database
Kriging_mn_t2 converted to point
Kriging_mn_t2_point pushed to database
Kriging_mn_universal converted to point
Kriging_mn_universal_point pushed to database
